In [1]:
import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
Independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [9]:
Independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
Independent.shape

(400, 3)

In [11]:
Dependent=dataset[['Purchased']]

In [12]:
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split,GridSearchCV
x_train,x_test,y_train,y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [58]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit_transform(x_train)
sc.transform(x_test)

array([[-0.77101313,  0.49720103,  0.99288247],
       [ 0.0133054 , -0.57280368, -1.00716855],
       [-0.28081405,  0.15017248,  0.99288247],
       [-0.77101313,  0.26584866, -1.00716855],
       [-0.28081405, -0.57280368, -1.00716855],
       [-1.06513258, -1.44037507,  0.99288247],
       [-0.67297331, -1.5849703 , -1.00716855],
       [-0.18277423,  2.14558666,  0.99288247],
       [-1.94749093, -0.05226085, -1.00716855],
       [ 0.89566375, -0.775237  ,  0.99288247],
       [-0.77101313, -0.60172273,  0.99288247],
       [-0.96709276, -0.42820845, -1.00716855],
       [-0.08473441, -0.42820845,  0.99288247],
       [ 0.11134522,  0.20801057,  0.99288247],
       [-1.7514113 ,  0.46828198,  0.99288247],
       [-0.5749335 ,  1.36477242, -1.00716855],
       [-0.08473441,  0.20801057,  0.99288247],
       [-1.84945111,  0.43936294,  0.99288247],
       [ 1.67998229,  1.74072002, -1.00716855],
       [-0.28081405, -1.38253697,  0.99288247],
       [-0.28081405, -0.65956082, -1.007

In [15]:
from sklearn.ensemble import RandomForestClassifier
param_grid={'n_estimators':[10,20,30],'criterion':['gini','entropy','log_loss'],'max_features':['sqrt','log2']}
grid=GridSearchCV(RandomForestClassifier(),param_grid,verbose=3,n_jobs=-1,refit=True)

In [16]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\91805\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 20, 30]},
             verbose=3)

In [17]:
y_pred=grid.predict(x_test)

In [18]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score

In [19]:
cm=confusion_matrix(y_test,y_pred)

In [20]:
print(cm)

[[73  6]
 [ 5 36]]


In [21]:
clf_rpt=classification_report(y_test,y_pred)

In [22]:
print (clf_rpt)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        79
           1       0.86      0.88      0.87        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [23]:
roc_score=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

In [24]:
roc_score

0.9518369867242976

In [25]:
re=grid.cv_results_

In [26]:
print("Best parameters are ", grid.best_params_)

Best parameters are  {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 20}


In [27]:
table=pd.DataFrame.from_dict(re)

In [28]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.059045,0.007984,0.011314,0.001812,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.839286,0.839286,0.821429,0.928571,0.910714,0.867857,0.043154,14
1,0.105425,0.009810,0.011969,0.001399,gini,sqrt,20,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.875000,0.857143,0.839286,0.946429,0.910714,0.885714,0.038465,2
2,0.135354,0.004604,0.015947,0.003777,gini,sqrt,30,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.875000,0.839286,0.839286,0.910714,0.928571,0.878571,0.036422,6
3,0.052046,0.004971,0.011058,0.000707,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.875000,0.839286,0.821429,0.875000,0.857143,0.853571,0.020825,18
4,0.088140,0.009230,0.010738,0.001501,gini,log2,20,"{'criterion': 'gini', 'max_features': 'log2', ...",0.892857,0.892857,0.839286,0.910714,0.946429,0.896429,0.034626,1
5,0.132443,0.008802,0.014355,0.002566,gini,log2,30,"{'criterion': 'gini', 'max_features': 'log2', ...",0.875000,0.839286,0.821429,0.928571,0.964286,0.885714,0.053690,2
6,0.056805,0.005620,0.010224,0.001537,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.875000,0.839286,0.839286,0.892857,0.910714,0.871429,0.028571,11
7,0.100389,0.003405,0.013656,0.003086,entropy,sqrt,20,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.821429,0.875000,0.839286,0.928571,0.928571,0.878571,0.044320,6
8,0.137624,0.007210,0.012477,0.001722,entropy,sqrt,30,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.875000,0.875000,0.821429,0.910714,0.946429,0.885714,0.041650,5
9,0.053602,0.005848,0.010844,0.002444,entropy,log2,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.857143,0.785714,0.821429,0.910714,0.892857,0.853571,0.045737,17
